In [1]:
## Proyecto individual, carga de librerías

import pandas as pd
from pandas import json_normalize
import json
import numpy as np

In [2]:
df = pd.read_csv('movies_dataset.csv')


C:\Users\Usuario\AppData\Local\Temp\ipykernel_7520\3347023233.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_dataset.csv')


In [3]:
df.head()


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
## Revisión de la información

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [5]:
# Resetear los index luego de borrar filas
df = df.reset_index(drop=True)


In [6]:
## Cambiar datos de objet a int64
df["budget"]=pd.to_numeric(df["budget"],errors='coerce')
df["id"]=pd.to_numeric(df["id"],errors='coerce')

In [7]:
##borrando columnas que no voy a utilizar
df = df.drop("video",axis=1)
df = df.drop("imdb_id",axis=1)
df = df.drop("adult",axis=1)
df = df.drop("poster_path",axis=1)
df = df.drop("homepage",axis=1)

In [8]:
# la función obtener_valor verifica si el valor proporcionado es un diccionario válido o una cadena que representa un diccionario. Si es un diccionario válido, devuelve el valor correspondiente a la clave especificada; de lo contrario, devuelve None. Esto nos permite extraer los valores deseados del diccionario almacenado en la columna "belongs_to_collection" y asignarlos a columnas separadas en el DataFrame.


def obtener_valor(diccionario, clave):
    if pd.isnull(diccionario):
        return None
    if not isinstance(diccionario, dict):
        try:
            diccionario = json.loads(diccionario.replace("'", "\""))
        except (json.JSONDecodeError, AttributeError):
            return None
    if isinstance(diccionario, dict):
        return diccionario.get(clave)
    else:
        return None


# Aplicar la función a la columna "belongs_to_collection"
df['id_btc'] = df['belongs_to_collection'].apply(
    lambda x: obtener_valor(x, 'id'))
df['name_btc'] = df['belongs_to_collection'].apply(
    lambda x: obtener_valor(x, 'name'))
df['poster_btc'] = df['belongs_to_collection'].apply(
    lambda x: obtener_valor(x, 'poster_path'))
df['backdrop_btc'] = df['belongs_to_collection'].apply(
    lambda x: obtener_valor(x, 'backdrop_path'))

# Imprimir los valores obtenidos
result = df[['id_btc', 'name_btc', 'poster_btc', 'backdrop_btc']]

result

,id_btc,name_btc,poster_btc,backdrop_btc
0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,NaN,None,None,None
2,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,NaN,None,None,None
4,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
...,...,...,...,...
45461,NaN,None,None,None
45462,NaN,None,None,None
45463,NaN,None,None,None
45464,NaN,None,None,None


In [9]:
df = df.drop('belongs_to_collection', axis=1)
df = df.drop('poster_btc', axis=1)
df = df.drop('backdrop_btc', axis=1)


In [10]:
df.head(5)


,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,id_btc,name_btc
0,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,10194.0,Toy Story Collection
1,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844.0,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,None
2,0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602.0,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,119050.0,Grumpy Old Men Collection
3,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357.0,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,None
4,0.0,"[{'id': 35, 'name': 'Comedy'}]",11862.0,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,96871.0,Father of the Bride Collection


In [11]:
def desanidar_generos(genres):
    """
    Desanida los géneros contenidos en una lista de diccionarios.

    Args:
        genres (list): Lista de diccionarios que representan los géneros.

    Returns:
        tuple: Una tupla que contiene dos listas: genero_id y genero_nombre.

    """
    genero_id = []
    genero_nombre = []
    for genero in genres:
        genero_id.append(genero['id'])
        genero_nombre.append(genero['name'])
    return genero_id, genero_nombre


# Aplicar la función desanidar_generos a la columna 'genres'
generos_desanidados = df['genres'].apply(lambda x: desanidar_generos(eval(x)))

# Crear un nuevo DataFrame a partir de los géneros desanidados
nuevo_df = pd.DataFrame(generos_desanidados.tolist(), columns=[
                        'genero_id', 'genero_nombre'])

# Explotar las columnas para que cada fila tenga un solo género
nuevo_df = nuevo_df.apply(pd.Series.explode)

# Reiniciar el índice del DataFrame
nuevo_df = nuevo_df.reset_index(drop=True)

# Mostrar las primeras filas del nuevo DataFrame
nuevo_df.head()


,genero_id,genero_nombre
0,16,Animation
1,35,Comedy
2,10751,Family
3,12,Adventure
4,14,Fantasy


In [12]:
# Elimino columna genres
del df['genres']


In [13]:

# Unir el DataFrame original con las columnas 'genero_id' y 'genero_nombre' del nuevo DataFrame
df = df.join(nuevo_df[['genero_id', 'genero_nombre']])


In [14]:

# Mostrar las primeras filas del DataFrame
df.head(5)


,budget,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,...,spoken_languages,status,tagline,title,vote_average,vote_count,id_btc,name_btc,genero_id,genero_nombre
0,30000000.0,862.0,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,10194.0,Toy Story Collection,16,Animation
1,65000000.0,8844.0,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,None,35,Comedy
2,0.0,15602.0,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,119050.0,Grumpy Old Men Collection,10751,Family
3,16000000.0,31357.0,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,None,12,Adventure
4,0.0,11862.0,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,96871.0,Father of the Bride Collection,14,Fantasy


In [15]:
# Convertir la columna 'fecha' a tipo fecha
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Verificar el tipo de dato después de la conversión
print(df['release_date'].dtype)  # datetime64[ns]


datetime64[ns]


In [16]:
# Crear una nueva columna con solo el año
df['release_year'] = df['release_date'].dt.year
# Mostrar el DataFrame resultante
df.head(5)


,budget,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,...,status,tagline,title,vote_average,vote_count,id_btc,name_btc,genero_id,genero_nombre,release_year
0,30000000.0,862.0,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,...,Released,NaN,Toy Story,7.7,5415.0,10194.0,Toy Story Collection,16,Animation,1995.0
1,65000000.0,8844.0,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,...,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,None,35,Comedy,1995.0
2,0.0,15602.0,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,119050.0,Grumpy Old Men Collection,10751,Family,1995.0
3,16000000.0,31357.0,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,None,12,Adventure,1995.0
4,0.0,11862.0,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,96871.0,Father of the Bride Collection,14,Fantasy,1995.0


In [17]:
df = df.drop('original_title', axis=1)


In [18]:

# Eliminar los valores NaN de la columna fecha
df = df.dropna(subset=['release_year'])
# Mostrar el DataFrame resultante
df.head()


,budget,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,...,status,tagline,title,vote_average,vote_count,id_btc,name_btc,genero_id,genero_nombre,release_year
0,30000000.0,862.0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,...,Released,NaN,Toy Story,7.7,5415.0,10194.0,Toy Story Collection,16,Animation,1995.0
1,65000000.0,8844.0,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,...,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,None,35,Comedy,1995.0
2,0.0,15602.0,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,119050.0,Grumpy Old Men Collection,10751,Family,1995.0
3,16000000.0,31357.0,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,None,12,Adventure,1995.0
4,0.0,11862.0,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,96871.0,Father of the Bride Collection,14,Fantasy,1995.0


In [20]:

#Convertir en enteros los valores
df['release_year'] = df['release_year'].astype(int)
df['revenue'] = df['revenue'].astype(int)
df['runtime'] = df['runtime'].astype(int)
# Mostrar el DataFrame resultante
df.head()


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [21]:
# Definir una función para calcular el retorno de inversión
def calcular_retorno(row):
    revenue = row['revenue']
    budget = row['budget']

    # Verificar si hay datos faltantes o el presupuesto es igual a cero
    if pd.isnull(revenue) or pd.isnull(budget) or budget == 0:
        return 0

    # Calcular el retorno de inversión y redondearlo a dos decimales
    return round(revenue / budget, 2)


# Aplicar la función a cada fila del DataFrame para crear la columna "return"
df['return'] = df.apply(calcular_retorno, axis=1)


In [ ]:
df.head()

,budget,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,...,tagline,title,vote_average,vote_count,id_btc,name_btc,genero_id,genero_nombre,release_year,return
0,30000000.0,862.0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033,81,...,NaN,Toy Story,7.7,5415.0,10194.0,Toy Story Collection,16,Animation,1995,12.45
1,65000000.0,8844.0,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures,United States of America,1995-12-15,262797249,104,...,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,None,35,Comedy,1995,4.04
2,0.0,15602.0,en,A family wedding reignites the ancient feud be...,11.7129,Warner Bros.,United States of America,1995-12-22,0,101,...,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,119050.0,Grumpy Old Men Collection,10751,Family,1995,0.00
3,16000000.0,31357.0,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156,127,...,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,None,12,Adventure,1995,5.09
4,0.0,11862.0,en,Just when George Banks has recovered from his ...,8.387519,Sandollar Productions,United States of America,1995-02-10,76578911,106,...,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,96871.0,Father of the Bride Collection,14,Fantasy,1995,0.00


In [22]:
# Obtener la lista de encabezados de columnas
encabezados = df.columns.tolist()

# Mostrar la lista de encabezados de columnas
print(encabezados)


['budget', 'id', 'original_language', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'id_btc', 'name_btc', 'genero_id', 'genero_nombre', 'release_year', 'return']


In [23]:
import ast


In [24]:
# Get the most important production companie
def convert_company(text):
    production_company = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 1:
            production_company.append(i['name'])
        counter += 1
    return production_company


df['production_companies'] = df['production_companies'].apply(
    convert_company)


In [25]:
#Convierto el contenido de la columna en texto, luego elimino los corchetes y elimino las comillas 
df['production_companies'] = df['production_companies'].astype(str)

df['production_companies'] = df['production_companies'].str.replace(
    '[', '').str.replace(']', '')
df['production_companies'] = df['production_companies'].str.replace("'", '')


In [26]:
#Elimino los caracteres especiales de la columna de production countries

def convert_company(text):
    production_company = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 1:
            production_company.append(i['name'])
        counter += 1
    return production_company


df['production_countries'] = df['production_countries'].apply(
    convert_company)


In [27]:
# Convierto el contenido de la columna en texto, luego elimino los corchetes y elimino las comillas
df['production_countries'] = df['production_countries'].astype(str)

df['production_countries'] = df['production_countries'].str.replace(
    '[', '').str.replace(']', '')
df['production_countries'] = df['production_countries'].str.replace("'", '')


In [28]:
df["runtime"].unique()


array([8.100e+01, 1.040e+02, 1.010e+02, 1.270e+02, 1.060e+02, 1.700e+02,
       9.700e+01, 1.300e+02, 8.800e+01, 7.800e+01, 1.920e+02, 1.190e+02,
       1.780e+02, 1.360e+02, 9.800e+01, 9.000e+01, 1.030e+02, 1.050e+02,
       1.240e+02, 1.320e+02, 1.110e+02, 1.120e+02, 1.230e+02, 1.000e+02,
       1.080e+02, 9.900e+01, 1.290e+02, 5.000e+01, 8.900e+01, 1.210e+02,
       1.220e+02, 5.100e+01, 1.170e+02, 1.160e+02, 9.600e+01, 9.300e+01,
       9.500e+01, 1.150e+02, 1.370e+02, 9.200e+01, 8.500e+01, 9.100e+01,
       1.750e+02, 8.700e+01, 7.000e+01, 1.180e+02, 1.020e+02, 1.350e+02,
       1.100e+02, 1.770e+02, 1.140e+02, 1.130e+02, 8.600e+01, 1.250e+02,
       1.400e+02, 1.390e+02, 8.200e+01, 8.300e+01, 1.280e+02, 1.090e+02,
       8.000e+01, 1.340e+02, 1.070e+02, 9.400e+01, 1.450e+02, 7.300e+01,
       0.000e+00, 7.700e+01, 1.710e+02, 1.330e+02, 1.540e+02, 1.620e+02,
       1.420e+02, 1.260e+02, 1.410e+02, 1.800e+02, 1.910e+02, 6.100e+01,
       8.400e+01, 1.440e+02, 1.310e+02, 1.380e+02, 

In [29]:
df['runtime'] = df['runtime'].fillna(0)


In [30]:

def contar_mes(df, mes):
    # Convertir la columna release_date a tipo datetime
    df['release_date'] = pd.to_datetime(df['release_date'])

    # Obtener el nombre del mes en español
    df['mes'] = df['release_date'].dt.strftime('%B').str.capitalize()

    # Contar los meses y devolver el resultado
    conteo_meses = df['mes'].value_counts().to_dict()

    # Devolver el conteo del mes solicitado
    return conteo_meses.get(mes.capitalize(), 0)




In [31]:

# Ejemplo de uso
cantidad_pelis = contar_mes(df, 'diciembre')
print(f"La cantidad de películas estrenadas en enero es: {cantidad_pelis}")


La cantidad de películas estrenadas en enero es: 0


In [32]:
tipos_de_dato = df.dtypes

print(tipos_de_dato)


budget                         float64
id                             float64
original_language               object
overview                        object
popularity                      object
production_companies            object
production_countries            object
release_date            datetime64[ns]
revenue                          int32
runtime                        float64
spoken_languages                object
status                          object
tagline                         object
title                           object
vote_average                   float64
vote_count                     float64
id_btc                         float64
name_btc                        object
genero_id                       object
genero_nombre                   object
release_year                     int32
return                         float64
mes                             object
dtype: object


In [33]:
#Función para devolver la cantidad de películas en un mes específico 

def antidad_filmaciones_mes(data, mes):
    # Convertir la columna "release_date" a tipo datetime si no está en ese formato
    data['release_date'] = pd.to_datetime(data['release_date'])
    # Filtrar los datos por el mes especificado en español
    data_filtrado = data[data['release_date'].dt.month_name(
        locale='es') == mes]
    # Obtener la cantidad de registros que coinciden
    cantidad = len(data_filtrado)
    return cantidad


In [34]:
#uso de la función del mes
cantidad_depelis = antidad_filmaciones_mes(df, 'Diciembre')
print(f"La cantidad de películas estrenadas en el mes es {cantidad_depelis}")

La cantidad de películas estrenadas en el mes es 3786


In [35]:
# Función para devolver la cantidad de películas en un dìa específico

def cantidad_filmaciones_dia(data, dia):
    # Filtrar los datos por el día de la semana especificado en español
    data_filtrado = data[data['release_date'].dt.day_name(locale='es') == dia]
    # Obtener la cantidad de registros que coinciden
    cantidad = len(data_filtrado)
    return cantidad


In [36]:
# uso de la función del día
cantidad_dia = cantidad_filmaciones_dia(df, 'Martes')
print(f"La cantidad de películas estrenadas en el dìa es {cantidad_dia}")


La cantidad de películas estrenadas en el dìa es 4641


In [37]:
#consulta del score con el año de la película

def score_titulo(title):
    # Reemplaza 'tu_archivo.csv' con el nombre de tu archivo de datos
    pelicula = df[df['title'] == title]
    if pelicula.empty:
        return "No se encontró información para la película: " + title
    else:
        release_year = pelicula['release_year'].values[0]
        score = pelicula['popularity'].values[0]
        return f"La película '{title}' fue estrenada en el año {release_year} y tiene un puntaje de {score}"



In [38]:

# Ejemplo de uso:
resultado = score_titulo('Toy Story')
print(resultado)


La película 'Toy Story' fue estrenada en el año 1995 y tiene un puntaje de 21.946943


In [39]:
def votos_titulo(titulo_de_la_filmacion):
    data = df
    pelicula = data[data['title'] == titulo_de_la_filmacion]
    if pelicula.empty:
        return "No se encontró información para la película: " + titulo_de_la_filmacion
    else:
        votos = pelicula['vote_count'].values[0]
        promedio = pelicula['vote_average'].values[0]
        if votos >= 2000:
            return f"La película '{titulo_de_la_filmacion}' fue estrenada en el año {pelicula['release_year'].values[0]}. La misma cuenta con un total de {votos} valoraciones, con un promedio de {promedio}"
        else:
            return f"La película '{titulo_de_la_filmacion}' no cumple con la condición de tener al menos 2000 valoraciones."


In [40]:
resultado = votos_titulo('Toy Story')
print(resultado)


La película 'Toy Story' fue estrenada en el año 1995. La misma cuenta con un total de 5415.0 valoraciones, con un promedio de 7.7


In [ ]:
df.to_csv('data_movies_fin.csv', index=False)

In [41]:
df['id'] = df['id'].astype(int)
df.head()

,budget,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,...,title,vote_average,vote_count,id_btc,name_btc,genero_id,genero_nombre,release_year,return,mes
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033,81.0,...,Toy Story,7.7,5415.0,10194.0,Toy Story Collection,16,Animation,1995,12.45,October
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures,United States of America,1995-12-15,262797249,104.0,...,Jumanji,6.9,2413.0,NaN,None,35,Comedy,1995,4.04,December
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.7129,Warner Bros.,United States of America,1995-12-22,0,101.0,...,Grumpier Old Men,6.5,92.0,119050.0,Grumpy Old Men Collection,10751,Family,1995,0.00,December
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156,127.0,...,Waiting to Exhale,6.1,34.0,NaN,None,12,Adventure,1995,5.09,December
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,Sandollar Productions,United States of America,1995-02-10,76578911,106.0,...,Father of the Bride Part II,5.7,173.0,96871.0,Father of the Bride Collection,14,Fantasy,1995,0.00,February


In [43]:
df_peliculas = df
df_creditos = pd.read_csv('creditos_fin.csv')

In [44]:
df_unido = pd.merge(df_peliculas, df_creditos, on='id')

In [45]:
df_unido.to_csv('data_movies_unido.csv', index=False)